<a href="https://colab.research.google.com/github/LonnyCox55/MSDS692_ParasiticInfectionCNN/blob/main/src/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lonny Cox-Lauf
#MSDS692 Practicum I
#CNN to Classify Parasitic Infection Type using Microscopic Images

This is the main Python notebook file that will clean the image data files, train, and evaluate the CNN models to classify types of parasitic eggs through microscopy.

In [1]:
# Import all necessary TensorFlow libraries
from tensorflow.keras import backend as K
from tensorflow.keras import backend
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Library for data augmentation
from tensorflow.keras.preprocessing import image

In [14]:
# Import all other necessary libraries for the project
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from IPython.display import display # Library to help view images
from PIL import Image # Library to help view images
import os, shutil # Library for navigating files
from google.colab import drive # Library to mount google drives
import glob

In [ ]:
# TO DO: Import all the CNN model instantiation functions


In [5]:
# Seed the random number generator for consistency
np.random.seed(55)

In [3]:
# Mount the Google Drive to access the image data .zip files
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# Un-zip all the training and testing image data files into Google Drive
!unzip gdrive/MyDrive/MSDS692_Practicum_I/data/Chula-ParasiteEgg-11.zip > /dev/null
#!unzip gdrive/MyDrive/MSDS692_Practicum_I/data/Chula-ParasiteEgg-11_test.zip > /dev/null TO DO: should I use this test data? It's unlabeled...

In [16]:
# Specify the base directory where all the images are located
baseDir = '/content/Chula-ParasiteEgg-11/Chula-ParasiteEgg-11/data'

/bin/bash: line 1: cd: baseDir: No such file or directory
/content


In [41]:
# Get a list of all the image filenames
#allFiles = os.listdir(baseDir)
testLonny = glob.glob("Ascaris*lumbricoides*", root_dir = baseDir)

In [43]:
0.2 * 0.8

0.16000000000000003

In [44]:
# Iterate through all 11 classifications of parasitic eggs
for eggClass in ("Hymenolepis*nana*", "Ascaris*lumbricoides*", "Enterobius*vermicularis*", "Capillaria*philippinensis*", "Hymenolepis*diminuta*", "Paragonimus*spp*", "Trichuris*trichiura*", "Fasciolopsis*buski*", "Opisthorchis*viverrine*", "Taenia*spp*", "Hookworm*"):
  eggFiles = glob.glob(eggClass, root_dir = baseDir) # Find all egg image files that match the current class
  eggClass.replace("*", "_")                         # Replace * with underscore for the new subdirectory names
  os.makedirs(baseDir + "/" + eggClass)              # Create new validation directory
  np.random.shuffle(eggFiles)                        # Randomize the files
  numTestFiles = int(0.2*len(eggFiles))              # Find 20% of the number of files for the test set
  testFiles = eggFiles[-numTestFiles:]               # Create subset of the data files for the test set

  # Move each test file to the respective test set directory
  for fname in testFiles:
    shutil.move(baseDir + "/" + fname,                       # Source
                baseDir + "/" + eggClass + "/test/" + fname) # Destination

  numValFiles = int(0.16*len(eggFiles))             # Find 20% of the remaining (training set) files for the validation set
  valFiles = eggFiles[-(numTestFiles+numValFiles):] # Create subset of the data files for the validation set

  # Move each validation file to the respective validation set directory
  for fname in valFiles:
    shutil.move(baseDir + "/" + fname,                      # Source
                baseDir + "/" + eggClass + "/val/" + fname) # Destination

  # Create subset of the data files for the training set
  numTrainFiles = len(eggFiles) - numTestFiles - numValFiles
  trainFiles = eggFiles[:numTrainFiles]

FileNotFoundError: [Errno 2] No such file or directory: '/content/Chula-ParasiteEgg-11/Chula-ParasiteEgg-11/data/Hymenolepis*nana*/test/Hymenolepis nana_1000.jpg'